In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

import pytry

In [ ]:
class PatternInterpolationTrial(pytry.Trial):
    def params(self):
        self.param('number of neurons', n_neurons=2000),
        
    def evaluate(self, q): #function to fill in missing data points by interpolating
        data = pd.read_csv("pinsoro-2017-06-20-145454904791-small.csv", low_memory=False)

        x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
        labs = np.array(data.iloc[:,218]).astype(str) #array of labels (purple child annotations, engagement)
        a = []
        for i in range(x.shape[1]):
            y = pd.Series(x[:,i])
            z = y.interpolate(limit_direction='both')
            a.append(z)
        a = pd.DataFrame(a)
        a = a.dropna()
        a = np.array(a).T

        dt = 0.001

        def extract_pattern(start, end): #function to extract consecutive frames with the same label 
            pattern = np.array(a[start:end,:]).astype(float)
            frames = np.array(data.iloc[start:end,9]).astype(int)

            good_indices = frames != -1
            frames = frames[good_indices]
            pattern = pattern[good_indices]

            fps = 30.0
            t_sample = (frames - frames[0])/fps

            t = np.arange(int(t_sample[-1]/dt))*dt

            result = []
            for i in range(pattern.shape[1]):       
                p = np.interp(t, t_sample, pattern[:,i])
                result.append(p)
            result = np.array(result).T

            return t, result

        start=[]
        start.append(0)
        end=[]
        label = []
        for i in range(1, (len(labs)-1)):
            if labs[i]!=labs[i-1]:
                start.append(i)
            if labs[i]!=labs[i+1]:
                end.append(i)
                label.append(labs[i])


        t_noplay=[]
        p_noplay=[]
        t_goal=[]
        p_goal=[]
        t_aim=[]
        p_aim=[]

        for i in range(1,(len(start)-1)):
            if label[i]==('noplay'):
                ti, pi = extract_pattern(start[i], end[i])
                t_noplay.append(ti)
                p_noplay.append(pi)
            if label[i]==('goaloriented'):
                ti, pi = extract_pattern(start[i], end[i])
                t_goal.append(ti)
                p_goal.append(pi)
            if label[i]==('aimless'):
                ti, pi = extract_pattern(start[i], end[i])
                t_aim.append(ti)
                p_aim.append(pi)

                
############ randomly split data 80/20 into training and testing sets ############ 
        random.shuffle(p_goal)
        goal_train = p_goal[:(int(len(p_goal)*0.8))] 
        goal_test = p_goal[(int(len(p_goal)*0.8)):]

        random.shuffle(p_noplay)
        noplay_train = p_noplay[:(int(len(p_noplay)*0.8))]
        noplay_test = p_noplay[(int(len(p_noplay)*0.8)):]

############ create pca model and convert each pattern into single dimension (first PCA construct) ############ 
        train_all = np.vstack(goal_train+noplay_train)
        pca_model = PCA(n_components=2).fit(train_all)

        goal_patterns1 = {}
        goal_patterns2 = {}
        i=0
        for gp in goal_train:
            goal_patterns1[i] = pca_model.transform(gp)[:,0]
            goal_patterns2[i] = pca_model.transform(gp)[:,1]
            i+=1

        noplay_patterns1 = {}
        noplay_patterns2 = {}
        i=0
        for npp in noplay_train:
            noplay_patterns1[i] = pca_model.transform(npp)[:,0]
            noplay_patterns2[i] = pca_model.transform(npp)[:,1]
            i+=1

        goal_patterns1 = [(goal_patterns1[key]) for key in goal_patterns1.keys()]
        goal_patterns2 = [(goal_patterns2[key]) for key in goal_patterns2.keys()]
        noplay_patterns1 = [(noplay_patterns1[key]) for key in noplay_patterns1.keys()]
        noplay_patterns2 = [(noplay_patterns2[key]) for key in noplay_patterns2.keys()]

        goal_patterns = np.vstack([goal_patterns1[0], goal_patterns2[0]]).T
        for gp in range(1,len(goal_train)):
            goal_patterns = np.append(goal_patterns, (np.vstack([goal_patterns1[gp], goal_patterns2[gp]]).T), axis=0) 
        noplay_patterns = np.vstack([noplay_patterns1[0], noplay_patterns2[0]]).T
        for npp in range(1,len(noplay_train)):
            noplay_patterns = np.append(noplay_patterns, (np.vstack([noplay_patterns1[npp], noplay_patterns2[npp]]).T), axis=0)  

############ TRAINING WITH 80% ############ 
        D = 2

        theta = 0.5
        net = nengo.Network()
        with net:
            process = nengo.processes.WhiteSignal(period=100., high=3.0, y0=0)
            #rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=q.n_neurons, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))
            rw = []
            for i in range(D):
                rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                          process=process, 
                                                          neuron_type=nengo.LIFRate()))

        s_window = int(theta/dt)
        s_pattern = len(min(goal_patterns1, key=len))

        t_window = np.linspace(0, 1, s_window)
        inv_basis = rw[0].inverse_basis(t_window)

        eval_points=[]
        target = []
        for i in range(s_pattern):
            eval_points.append(np.dot(inv_basis, np.roll(goal_patterns, i)[:s_window]).T)
            target.append([1])
        for i in range(s_pattern):  
            eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns, i)[:s_window]).T)
            target.append([-1])
        eval_points = np.array(eval_points).reshape(len(eval_points), -1)

############ TEST ON TRAINING PATTERNS ############ 

        with net:

            pool = nengo.Ensemble(n_neurons=3000, dimensions=eval_points.shape[1],
                                  neuron_type=nengo.LIFRate(), seed=1)

            start = 0
            for r in rw:
                nengo.Connection(r.state, pool[start:start+r.state.size_out])
                start += r.state.size_out

            result = nengo.Node(None, size_in=1)
            dec_conn = nengo.Connection(pool, result,
                             eval_points=eval_points, scale_eval_points=False,
                             function=target, synapse=0.1)

        train_all1 = np.array(goal_patterns1[0])
        for gp in range(1,(len(goal_patterns1))):
            train_all1 = np.hstack([train_all1,goal_patterns1[gp]])
        for npp in range(0,(len(noplay_patterns1))):
            train_all1 = np.hstack([train_all1,noplay_patterns1[npp]])

        train_all2 = np.array(goal_patterns2[0])
        for gp in range(1,(len(goal_patterns2))):
            train_all2 = np.hstack([train_all2,goal_patterns2[gp]])
        for npp in range(0,(len(noplay_patterns2))):
            train_all2 = np.hstack([train_all2,noplay_patterns2[npp]])


        train_all = np.vstack([train_all1, train_all2])

        presentation_time_train=10/len(train_all.T)

        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(train_all, presentation_time=presentation_time_train))
            for i in range(D):
                nengo.Connection(stim[i], rw[i].input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
            p_pool = nengo.Probe(pool)
        sim = nengo.Simulator(model)
        sim.run(10)

############ GET RESULTS ############ 

        goal_train_len = {}
        goal_train_len[0] = int(len(goal_patterns1[0])*presentation_time_train/dt)
        for i in range(1,len(goal_train)):
            goal_train_len[i] = goal_train_len[i-1]+int(len(goal_patterns1[i])*presentation_time_train/dt)

        goal_train_means = {}
        goal_train_means[0] = np.mean(sim.data[p_result][0:goal_train_len[0]])
        for i in range(1,len(goal_train)):
            goal_train_means[i] = np.mean(sim.data[p_result][goal_train_len[i-1]:goal_train_len[i]])

        noplay_train_len = {}
        noplay_train_len[0] = int(len(noplay_patterns1[0])*presentation_time_train/dt)
        for i in range(1,len(noplay_train)):
            noplay_train_len[i] = noplay_train_len[i-1]+int(len(noplay_patterns1[i])*presentation_time_train/dt)

        noplay_train_means = {}
        noplay_train_means[0] = np.mean(sim.data[p_result][goal_train_len[len(goal_train_len)-1]:noplay_train_len[0]+goal_train_len[len(goal_train_len)-1]])
        for i in range(1,len(noplay_train)):
            noplay_train_means[i] = np.mean(sim.data[p_result][noplay_train_len[i-1]:noplay_train_len[i]])

        for j in range(len(goal_patterns1)):
            key_j = 'gtr_{}'.format(j)  
            goal_train_means[key_j] = goal_train_means.pop(j)

        for j in range(len(noplay_patterns1)):
            key_j = 'ntr_{}'.format(j)  
            noplay_train_means[key_j] = noplay_train_means.pop(j)

############ EXTRACT 20% TEST PATTERNS ############ 

        goal_test_patterns1 = {}
        goal_test_patterns2 = {}
        i=0
        for gp in goal_test:
            goal_test_patterns1[i] = pca_model.transform(gp)[:,0]
            goal_test_patterns2[i] = pca_model.transform(gp)[:,1]
            i+=1

        noplay_test_patterns1 = {}
        noplay_test_patterns2 = {}
        i=0
        for npp in noplay_test:
            noplay_test_patterns1[i] = pca_model.transform(npp)[:,0]
            noplay_test_patterns2[i] = pca_model.transform(npp)[:,1]
            i+=1

        goal_test_patterns1 = [(goal_test_patterns1[key]) for key in goal_test_patterns1.keys()]
        goal_test_patterns2 = [(goal_test_patterns2[key]) for key in goal_test_patterns2.keys()]
        noplay_test_patterns1 = [(noplay_test_patterns1[key]) for key in noplay_test_patterns1.keys()]
        noplay_test_patterns2 = [(noplay_test_patterns2[key]) for key in noplay_test_patterns2.keys()]

        goal_test_patterns = np.vstack([goal_test_patterns1[0], goal_test_patterns2[0]]).T
        for gp in range(1,len(goal_test)):
            goal_test_patterns = np.append(goal_test_patterns, (np.vstack([goal_test_patterns1[gp], goal_test_patterns2[gp]]).T), axis=0) 
        noplay_test_patterns = np.vstack([noplay_test_patterns1[0], noplay_test_patterns2[0]]).T
        for npp in range(1,len(noplay_test)):
            noplay_test_patterns = np.append(noplay_test_patterns, (np.vstack([noplay_test_patterns1[npp], noplay_test_patterns2[npp]]).T), axis=0) 


        test_all1 = np.array(goal_test_patterns1[0])
        for gp in range(1,(len(goal_test_patterns1))):
            test_all1 = np.hstack([test_all1,goal_test_patterns1[gp]])
        for npp in range(0,(len(noplay_test_patterns1))):
            test_all1 = np.hstack([test_all1,noplay_test_patterns1[npp]])

        test_all2 = np.array(goal_test_patterns2[0])
        for gp in range(1,(len(goal_test_patterns2))):
            test_all2 = np.hstack([test_all2,goal_test_patterns2[gp]])
        for npp in range(0,(len(noplay_test_patterns2))):
            test_all2 = np.hstack([test_all2,noplay_test_patterns2[npp]])


        test_all = np.vstack([test_all1, test_all2])

        presentation_time_test=10/len(test_all.T)

        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(test_all, presentation_time=presentation_time_test))
            for i in range(D):
                nengo.Connection(stim[i], rw[i].input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
            p_pool = nengo.Probe(pool)
        sim = nengo.Simulator(model)
        sim.run(10)

############ GET RESULTS ############ 

        goal_test_len = {}
        goal_test_len[0] = int(len(goal_test_patterns1[0])*presentation_time_test/dt)
        for i in range(1,len(goal_test)):
            goal_test_len[i] = goal_test_len[i-1]+int(len(goal_test_patterns1[i])*presentation_time_test/dt)

        goal_test_means = {}
        goal_test_means[0] = np.mean(sim.data[p_result][0:goal_test_len[0]])
        for i in range(1,len(goal_test)):
            goal_test_means[i] = np.mean(sim.data[p_result][goal_test_len[i-1]:goal_test_len[i]])

        noplay_test_len = {}
        noplay_test_len[0] = int(len(noplay_test_patterns1[0])*presentation_time_test/dt)
        for i in range(1,len(noplay_test)):
            noplay_test_len[i] = noplay_test_len[i-1]+int(len(noplay_test_patterns1[i])*presentation_time_test/dt)

        noplay_test_means = {}
        noplay_test_means[0] = np.mean(sim.data[p_result][goal_test_len[len(goal_test_len)-1]:noplay_test_len[0]+goal_test_len[len(goal_test_len)-1]])
        for i in range(1,len(noplay_test)):
            noplay_test_means[i] = np.mean(sim.data[p_result][noplay_test_len[i-1]:noplay_test_len[i]])

        for j in range(len(goal_test_patterns1)):
            key_j = 'gtst_{}'.format(j)  
            goal_test_means[key_j] = goal_test_means.pop(j)

        for j in range(len(noplay_test_patterns1)):
            key_j = 'ntst_{}'.format(j)  
            noplay_test_means[key_j] = noplay_test_means.pop(j)


################# TEST ON AIMLESS PATTERNS #################

        random.shuffle(p_aim)

        aimless_patterns1 = {}
        aimless_patterns2 = {}
        i=0
        for ap in p_aim:
            aimless_patterns1[i] = pca_model.transform(ap)[:,0]
            aimless_patterns2[i] = pca_model.transform(ap)[:,1]
            i+=1


        test_aim1 = np.array(aimless_patterns1[0])
        for ap in range(1,(len(aimless_patterns1))):
            test_aim1 = np.hstack([test_aim1,aimless_patterns1[ap]])

        test_aim2 = np.array(aimless_patterns2[0])
        for ap in range(1,(len(aimless_patterns2))):
            test_aim2 = np.hstack([test_aim2,aimless_patterns2[ap]])

        test_aim = np.vstack([test_aim1, test_aim2])

        presentation_time_aim=10/len(test_aim.T)

        model = nengo.Network()
        model.networks.append(net)
        with model:    
            stim = nengo.Node(nengo.processes.PresentInput(test_aim, presentation_time=presentation_time_aim))
            for i in range(D):
                nengo.Connection(stim[i], rw[i].input, synapse=None)

            p_result = nengo.Probe(result)
            p_stim = nengo.Probe(stim)
            p_pool = nengo.Probe(pool)
        sim = nengo.Simulator(model)
        sim.run(10)

############ GET RESULTS ############ 

        aim_test_len = {}
        aim_test_len[0] = int(len(aimless_patterns1[0])*presentation_time_aim/dt)
        for i in range(1,len(p_aim)):
            aim_test_len[i] = aim_test_len[i-1]+int(len(aimless_patterns1[i])*presentation_time_aim/dt)

        aim_test_means = {}
        aim_test_means[0] = np.mean(sim.data[p_result][0:aim_test_len[0]])
        for i in range(1,len(p_aim)):
            aim_test_means[i] = np.mean(sim.data[p_result][aim_test_len[i-1]:aim_test_len[i]])

        for j in range(len(p_aim)):
            key_j = 'aim_{}'.format(j)  
            aim_test_means[key_j] = aim_test_means.pop(j)


############ SAVE DATA ############ 

        means = {**goal_train_means, **noplay_train_means, **goal_test_means, **noplay_test_means, **aim_test_means}

        return means

In [ ]:
PatternInterpolationTrial().run(data_dir='2D_debug')

In [ ]:
####################DEVELOPMENT AND DEBUG SCRIPT####################

%matplotlib inline
import matplotlib.pyplot as plt
import nengo
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score

import pytry

from sklearn.decomposition import PCA

data = pd.read_csv("pinsoro-2017-06-20-145454904791-small.csv", low_memory=False)

x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
labs = np.array(data.iloc[:,218]).astype(str) #array of labels (purple child annotations, engagement)
a = []
for i in range(x.shape[1]):
    y = pd.Series(x[:,i])
    z = y.interpolate(limit_direction='both')
    a.append(z)
a = pd.DataFrame(a)
a = a.dropna()
a = np.array(a).T

dt = 0.001

def extract_pattern(start, end): #function to extract consecutive frames with the same label 
    pattern = np.array(a[start:end,:]).astype(float)
    frames = np.array(data.iloc[start:end,9]).astype(int)

    good_indices = frames != -1
    frames = frames[good_indices]
    pattern = pattern[good_indices]

    fps = 30.0
    t_sample = (frames - frames[0])/fps

    t = np.arange(int(t_sample[-1]/dt))*dt

    result = []
    for i in range(pattern.shape[1]):       
        p = np.interp(t, t_sample, pattern[:,i])
        result.append(p)
    result = np.array(result).T

    return t, result

start=[]
start.append(0)
end=[]
label = []
for i in range(1, (len(labs)-1)):
    if labs[i]!=labs[i-1]:
        start.append(i)
    if labs[i]!=labs[i+1]:
        end.append(i)
        label.append(labs[i])


t_noplay=[]
p_noplay=[]
t_goal=[]
p_goal=[]
t_aim=[]
p_aim=[]

for i in range(1,(len(start)-1)):
    if label[i]==('noplay'):
        ti, pi = extract_pattern(start[i], end[i])
        t_noplay.append(ti)
        p_noplay.append(pi)
    if label[i]==('goaloriented'):
        ti, pi = extract_pattern(start[i], end[i])
        t_goal.append(ti)
        p_goal.append(pi)
    if label[i]==('aimless'):
        ti, pi = extract_pattern(start[i], end[i])
        t_aim.append(ti)
        p_aim.append(pi)

############ randomly split data 80/20 into training and testing sets ############ 
random.shuffle(p_goal)
goal_train = p_goal[:(int(len(p_goal)*0.8))] 
goal_test = p_goal[(int(len(p_goal)*0.8)):]

random.shuffle(p_noplay)
noplay_train = p_noplay[:(int(len(p_noplay)*0.8))]
noplay_test = p_noplay[(int(len(p_noplay)*0.8)):]

############ create pca model and convert each pattern into single dimension (first PCA construct) ############ 
train_all = np.vstack(goal_train+noplay_train)
pca_model = PCA(n_components=2).fit(train_all)

goal_patterns1 = {}
goal_patterns2 = {}
i=0
for gp in goal_train:
    goal_patterns1[i] = pca_model.transform(gp)[:,0]
    goal_patterns2[i] = pca_model.transform(gp)[:,1]
    i+=1

noplay_patterns1 = {}
noplay_patterns2 = {}
i=0
for npp in noplay_train:
    noplay_patterns1[i] = pca_model.transform(npp)[:,0]
    noplay_patterns2[i] = pca_model.transform(npp)[:,1]
    i+=1

goal_patterns1 = [(goal_patterns1[key]) for key in goal_patterns1.keys()]
goal_patterns2 = [(goal_patterns2[key]) for key in goal_patterns2.keys()]
noplay_patterns1 = [(noplay_patterns1[key]) for key in noplay_patterns1.keys()]
noplay_patterns2 = [(noplay_patterns2[key]) for key in noplay_patterns2.keys()]

goal_patterns = np.vstack([goal_patterns1[0], goal_patterns2[0]]).T
for gp in range(1,len(goal_train)):
    goal_patterns = np.append(goal_patterns, (np.vstack([goal_patterns1[gp], goal_patterns2[gp]]).T), axis=0) 
noplay_patterns = np.vstack([noplay_patterns1[0], noplay_patterns2[0]]).T
for npp in range(1,len(noplay_train)):
    noplay_patterns = np.append(noplay_patterns, (np.vstack([noplay_patterns1[npp], noplay_patterns2[npp]]).T), axis=0)  

############ TRAINING WITH 80% ############ 
D = 2

theta = 0.5
net = nengo.Network()
with net:
    process = nengo.processes.WhiteSignal(period=100., high=3.0, y0=0)
    #rw = nengolib.networks.RollingWindow(theta=theta, n_neurons=q.n_neurons, process=process, neuron_type=nengo.LIFRate(tau_rc=0.02, tau_ref=0.002, amplitude=1))
    rw = []
    for i in range(D):
        rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                  process=process, 
                                                  neuron_type=nengo.LIFRate()))

s_window = int(theta/dt)
s_pattern = len(min(goal_patterns1, key=len))

t_window = np.linspace(0, 1, s_window)
inv_basis = rw[0].inverse_basis(t_window)

eval_points=[]
target = []
for i in range(s_pattern):
    eval_points.append(np.dot(inv_basis, np.roll(goal_patterns, i)[:s_window]).T)
    target.append([1])
for i in range(s_pattern):  
    eval_points.append(np.dot(inv_basis, np.roll(noplay_patterns, i)[:s_window]).T)
    target.append([-1])
eval_points = np.array(eval_points).reshape(len(eval_points), -1)

############ TEST ON TRAINING PATTERNS ############ 

with net:

    pool = nengo.Ensemble(n_neurons=3000, dimensions=eval_points.shape[1],
                          neuron_type=nengo.LIFRate(), seed=1)

    start = 0
    for r in rw:
        nengo.Connection(r.state, pool[start:start+r.state.size_out])
        start += r.state.size_out

    result = nengo.Node(None, size_in=1)
    dec_conn = nengo.Connection(pool, result,
                     eval_points=eval_points, scale_eval_points=False,
                     function=target, synapse=0.1)

train_all1 = np.array(goal_patterns1[0])
for gp in range(1,(len(goal_patterns1))):
    train_all1 = np.hstack([train_all1,goal_patterns1[gp]])
for npp in range(0,(len(noplay_patterns1))):
    train_all1 = np.hstack([train_all1,noplay_patterns1[npp]])

train_all2 = np.array(goal_patterns2[0])
for gp in range(1,(len(goal_patterns2))):
    train_all2 = np.hstack([train_all2,goal_patterns2[gp]])
for npp in range(0,(len(noplay_patterns2))):
    train_all2 = np.hstack([train_all2,noplay_patterns2[npp]])


train_all = np.vstack([train_all1, train_all2])

presentation_time_train=10/len(train_all.T)

model = nengo.Network()
model.networks.append(net)
with model:    
    stim = nengo.Node(nengo.processes.PresentInput(train_all, presentation_time=presentation_time_train))
    for i in range(D):
        nengo.Connection(stim[i], rw[i].input, synapse=None)

    p_result = nengo.Probe(result)
    p_stim = nengo.Probe(stim)
    p_pool = nengo.Probe(pool)
sim = nengo.Simulator(model)
sim.run(10)

############ GET RESULTS ############ 

goal_train_len = {}
goal_train_len[0] = int(len(goal_patterns1[0])*presentation_time_train/dt)
for i in range(1,len(goal_train)):
    goal_train_len[i] = goal_train_len[i-1]+int(len(goal_patterns1[i])*presentation_time_train/dt)

goal_train_means = {}
goal_train_means[0] = np.mean(sim.data[p_result][0:goal_train_len[0]])
for i in range(1,len(goal_train)):
    goal_train_means[i] = np.mean(sim.data[p_result][goal_train_len[i-1]:goal_train_len[i]])

noplay_train_len = {}
noplay_train_len[0] = int(len(noplay_patterns1[0])*presentation_time_train/dt)
for i in range(1,len(noplay_train)):
    noplay_train_len[i] = noplay_train_len[i-1]+int(len(noplay_patterns1[i])*presentation_time_train/dt)

noplay_train_means = {}
noplay_train_means[0] = np.mean(sim.data[p_result][goal_train_len[len(goal_train_len)-1]:noplay_train_len[0]+goal_train_len[len(goal_train_len)-1]])
for i in range(1,len(noplay_train)):
    noplay_train_means[i] = np.mean(sim.data[p_result][noplay_train_len[i-1]:noplay_train_len[i]])

for j in range(len(goal_patterns1)):
    key_j = 'gtr_{}'.format(j)  
    goal_train_means[key_j] = goal_train_means.pop(j)

for j in range(len(noplay_patterns1)):
    key_j = 'ntr_{}'.format(j)  
    noplay_train_means[key_j] = noplay_train_means.pop(j)

############ EXTRACT 20% TEST PATTERNS ############ 

goal_test_patterns1 = {}
goal_test_patterns2 = {}
i=0
for gp in goal_test:
    goal_test_patterns1[i] = pca_model.transform(gp)[:,0]
    goal_test_patterns2[i] = pca_model.transform(gp)[:,1]
    i+=1

noplay_test_patterns1 = {}
noplay_test_patterns2 = {}
i=0
for npp in noplay_test:
    noplay_test_patterns1[i] = pca_model.transform(npp)[:,0]
    noplay_test_patterns2[i] = pca_model.transform(npp)[:,1]
    i+=1

goal_test_patterns1 = [(goal_test_patterns1[key]) for key in goal_test_patterns1.keys()]
goal_test_patterns2 = [(goal_test_patterns2[key]) for key in goal_test_patterns2.keys()]
noplay_test_patterns1 = [(noplay_test_patterns1[key]) for key in noplay_test_patterns1.keys()]
noplay_test_patterns2 = [(noplay_test_patterns2[key]) for key in noplay_test_patterns2.keys()]

goal_test_patterns = np.vstack([goal_test_patterns1[0], goal_test_patterns2[0]]).T
for gp in range(1,len(goal_test)):
    goal_test_patterns = np.append(goal_test_patterns, (np.vstack([goal_test_patterns1[gp], goal_test_patterns2[gp]]).T), axis=0) 
noplay_test_patterns = np.vstack([noplay_test_patterns1[0], noplay_test_patterns2[0]]).T
for npp in range(1,len(noplay_test)):
    noplay_test_patterns = np.append(noplay_test_patterns, (np.vstack([noplay_test_patterns1[npp], noplay_test_patterns2[npp]]).T), axis=0) 
    

test_all1 = np.array(goal_test_patterns1[0])
for gp in range(1,(len(goal_test_patterns1))):
    test_all1 = np.hstack([test_all1,goal_test_patterns1[gp]])
for npp in range(0,(len(noplay_test_patterns1))):
    test_all1 = np.hstack([test_all1,noplay_test_patterns1[npp]])

test_all2 = np.array(goal_test_patterns2[0])
for gp in range(1,(len(goal_test_patterns2))):
    test_all2 = np.hstack([test_all2,goal_test_patterns2[gp]])
for npp in range(0,(len(noplay_test_patterns2))):
    test_all2 = np.hstack([test_all2,noplay_test_patterns2[npp]])


test_all = np.vstack([test_all1, test_all2])

presentation_time_test=10/len(test_all.T)

model = nengo.Network()
model.networks.append(net)
with model:    
    stim = nengo.Node(nengo.processes.PresentInput(test_all, presentation_time=presentation_time_test))
    for i in range(D):
        nengo.Connection(stim[i], rw[i].input, synapse=None)

    p_result = nengo.Probe(result)
    p_stim = nengo.Probe(stim)
    p_pool = nengo.Probe(pool)
sim = nengo.Simulator(model)
sim.run(10)

############ GET RESULTS ############ 

goal_test_len = {}
goal_test_len[0] = int(len(goal_test_patterns1[0])*presentation_time_test/dt)
for i in range(1,len(goal_test)):
    goal_test_len[i] = goal_test_len[i-1]+int(len(goal_test_patterns1[i])*presentation_time_test/dt)

goal_test_means = {}
goal_test_means[0] = np.mean(sim.data[p_result][0:goal_test_len[0]])
for i in range(1,len(goal_test)):
    goal_test_means[i] = np.mean(sim.data[p_result][goal_test_len[i-1]:goal_test_len[i]])

noplay_test_len = {}
noplay_test_len[0] = int(len(noplay_test_patterns1[0])*presentation_time_test/dt)
for i in range(1,len(noplay_test)):
    noplay_test_len[i] = noplay_test_len[i-1]+int(len(noplay_test_patterns1[i])*presentation_time_test/dt)

noplay_test_means = {}
noplay_test_means[0] = np.mean(sim.data[p_result][goal_test_len[len(goal_test_len)-1]:noplay_test_len[0]+goal_test_len[len(goal_test_len)-1]])
for i in range(1,len(noplay_test)):
    noplay_test_means[i] = np.mean(sim.data[p_result][noplay_test_len[i-1]:noplay_test_len[i]])

for j in range(len(goal_test_patterns1)):
    key_j = 'gtst_{}'.format(j)  
    goal_test_means[key_j] = goal_test_means.pop(j)

for j in range(len(noplay_test_patterns1)):
    key_j = 'ntst_{}'.format(j)  
    noplay_test_means[key_j] = noplay_test_means.pop(j)

 
################# TEST ON AIMLESS PATTERNS #################

random.shuffle(p_aim)

aimless_patterns1 = {}
aimless_patterns2 = {}
i=0
for ap in p_aim:
    aimless_patterns1[i] = pca_model.transform(ap)[:,0]
    aimless_patterns2[i] = pca_model.transform(ap)[:,1]
    i+=1


test_aim1 = np.array(aimless_patterns1[0])
for ap in range(1,(len(aimless_patterns1))):
    test_aim1 = np.hstack([test_aim1,aimless_patterns1[ap]])

test_aim2 = np.array(aimless_patterns2[0])
for ap in range(1,(len(aimless_patterns2))):
    test_aim2 = np.hstack([test_aim2,aimless_patterns2[ap]])

test_aim = np.vstack([test_aim1, test_aim2])

presentation_time_aim=10/len(test_aim.T)

model = nengo.Network()
model.networks.append(net)
with model:    
    stim = nengo.Node(nengo.processes.PresentInput(test_aim, presentation_time=presentation_time_aim))
    for i in range(D):
        nengo.Connection(stim[i], rw[i].input, synapse=None)

    p_result = nengo.Probe(result)
    p_stim = nengo.Probe(stim)
    p_pool = nengo.Probe(pool)
sim = nengo.Simulator(model)
sim.run(10)

############ GET RESULTS ############ 

aim_test_len = {}
aim_test_len[0] = int(len(aimless_patterns1[0])*presentation_time_aim/dt)
for i in range(1,len(p_aim)):
    aim_test_len[i] = aim_test_len[i-1]+int(len(aimless_patterns1[i])*presentation_time_aim/dt)

aim_test_means = {}
aim_test_means[0] = np.mean(sim.data[p_result][0:aim_test_len[0]])
for i in range(1,len(p_aim)):
    aim_test_means[i] = np.mean(sim.data[p_result][aim_test_len[i-1]:aim_test_len[i]])

for j in range(len(p_aim)):
    key_j = 'aim_{}'.format(j)  
    aim_test_means[key_j] = aim_test_means.pop(j)


############ SAVE DATA ############ 

means = {**goal_train_means, **noplay_train_means, **goal_test_means, **noplay_test_means, **aim_test_means}

return means


In [ ]:
%reset